In [1]:
from model_management.Predict import main_predict
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
data_path = './historical/data/'

In [3]:
model_path = './trained_model_parameters/models_tobe_evaluated/model_2024-09-14_NP/'
#model_path = './trained_model_parameters/latest_model/'
main_predict(data_path=data_path, model_path=model_path,
             predict_days=60, save_prediction=True, update_prediction=True, avoid_training_set=True)

,日期,風力,太陽能,尖峰負載
0,2024-09-15,8.319832,0.000000,3188.794922
1,2024-09-16,21.445911,720.582865,3939.703857
2,2024-09-17,19.122545,789.586208,3348.211182
